In [6]:
import pandas as pd

auction_df = pd.read_csv('auction_df.csv')
hedonic_df = pd.read_csv('위경도_파싱.csv')

C:\Users\user\AppData\Local\Temp\ipykernel_20300\23274293.py:4: DtypeWarning: Columns (32,86,123) have mixed types. Specify dtype option on import or set low_memory=False.
  hedonic_df = pd.read_csv('위경도_파싱.csv')


In [7]:
df = hedonic_df
df = df[[col for col in df.columns if '표제부' not in col]]

In [10]:
hedonic_df.columns

Index(['PNU', '계약일자', '전월세구분', '전용면적(㎡)', '보증금(만원)', '보증금_증감률(%)', '월세금(만원)',
       '전세_환산가(만원)', '층', '계약기간_일수',
       ...
       '건축연령_sq', 'area_floor_inter', '매수자_법인_flag', 'age_buycorp_inter',
       'Predicted_ln', 'Predicted_Price', 'Actual_Price', 'Residual', '위도',
       '경도'],
      dtype='object', length=152)

In [8]:
auction_df.columns

Index(['사건번호', '종별', '외 건수', '경매현황', '감정가', '최저가', '매각가', '진행상태', '입찰일자', '연도',
       '타경', '개시결정일자', '층', '호', '건물명', 'SIDO_NM', 'SIGNGU_NM', 'EMD_NM',
       'LTNO_NM'],
      dtype='object')

In [9]:
import pandas as pd

def ltno_to_pnu(ltno, dong_code='1150010300'):
    # NaN, None, 'nan' 문자열 모두 처리
    if pd.isna(ltno) or str(ltno).lower() == 'nan':
        return None
    
    ltno = str(ltno)  # 문자열로 변환
    
    if '-' in ltno:
        main, sub = ltno.split('-')
    else:
        main, sub = ltno, '0'

    main = str(int(float(main))).zfill(4)  # float 거쳐서 변환
    sub  = str(int(float(sub))).zfill(4)

    land_type = '1'  # 일반 대지

    return dong_code + land_type + main + sub

auction_df['PNU'] = auction_df['LTNO_NM'].apply(ltno_to_pnu)

In [11]:
import pandas as pd
from datetime import datetime, timedelta


hedonic_df['PNU'] = hedonic_df['PNU'].astype(str)
auction_df['PNU'] = auction_df['PNU'].astype(str)

hedonic_df['계약일자'] = pd.to_datetime(hedonic_df['계약일자'], errors='coerce')
auction_df['개시결정일자'] = pd.to_datetime(auction_df['개시결정일자'], errors='coerce')
# hedonic_df에 경매 여부 컬럼 초기화
hedonic_df['경매_4년이내'] = 0

# 각 hedonic_df 행에 대해 확인
for idx, row in hedonic_df.iterrows():
    # 동일한 주소 찾기
    matched_auctions = auction_df[
        (auction_df['PNU'] == row['PNU'])  ]
    
    if not matched_auctions.empty:
        # 실거래일 이후의 경매만 필터링
        future_auctions = matched_auctions[matched_auctions['개시결정일자'] > row['계약일자']]
        
        if not future_auctions.empty:
   
            earliest_auction = future_auctions['개시결정일자'].min()
            days_diff = (earliest_auction - row['계약일자']).days
            
            # 4년(1460일) 이내 체크
            if days_diff <= 1460:
                hedonic_df.at[idx, '경매_4년이내'] = 1


In [12]:
hedonic_df['경매_4년이내'].value_counts()

경매_4년이내
0    18376
1      460
Name: count, dtype: int64

In [ ]:
hedonic_df.to_csv('D2_auction.csv', index=False)